In [115]:
import pandas as pd
import numpy as np
import re
import string

import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

from nltk import wordnet
# from wordnet import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
df = pd.read_csv("../data_hackathon/fake.csv")

# fake = fake[['uuid','text','site_url','type']]

### Copy pipeline  from here

In [111]:
df['text'] = df['text'].astype(str)
df['text'] = map(lambda x: x.lower(), df['text'])

#### Remove stop words

In [113]:
stop = stopwords.words('english')
df['text_sw'] = map(lambda x: ' '.join([word for word in x.split(' ') if word not in stop]), df['text'])

/root/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


In [114]:
df.head(2)

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,...,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type,text_sw,text_punc
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,print they should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,...,0.0,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,print pay back money plus interest. entire fam...,Print They pay back money plus interest The en...
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,why did attorney general loretta lynch plead t...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,...,0.0,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias,attorney general loretta lynch plead fifth? ba...,Why Did Attorney General Loretta Lynch Plead T...


#### Remove whitespaces and punctuations

In [145]:
# Retain only english characters, remove everything else

df['text_char_only'] = map(lambda x: re.sub('[^A-Za-z]+',' ',x).strip(), df['text_sw'])

df.head(2)

#### lemmatize, then stem

In [147]:
wnl = WordNetLemmatizer()

In [148]:
# Mapping from Penn Tag to Morphy Tag - Morphy tagger is used in WordNetLemmatizer
def penn2morphy(penntag, returnNone=False):
    morphy_tag = {'NN':wordnet.wordnet.NOUN, 'JJ':wordnet.wordnet.ADJ,
                   'VB':wordnet.wordnet.VERB, 'RB':wordnet.wordnet.ADV}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n'

In [162]:
def pos_and_lemmatize(x):
    try:
        tagged_text = pos_tag(x.split(" "))
        lemmatized_text = []
        for token_postag in tagged_text:
            lemmatized_text.append(wnl.lemmatize(token_postag[0],penn2morphy(token_postag[1])))
        lemmatized_text = " ".join(lemmatized_text)
        return lemmatized_text
    except:
        return x

In [163]:
df['text_lemmatized'] = map(lambda x: pos_and_lemmatize(x), df['text_char_only'])

In [ ]:
# sno = nltk.stem.SnowballStemmer('english')
# sno.stem('grows')
# 'grow'
# sno.stem('leaves')
# 'leav'
# sno.stem('fairly')
# 'fair'

In [ ]:
df.he

### Copy pipeline till here

#### Create bigrams, trigrams

In [173]:
from nltk.util import ngrams
from nltk import word_tokenize

In [207]:
def return_n_grams(x):
    bigrams = ngrams(word_tokenize(a),2)
    trigrams = ngrams(word_tokenize(a),3)
    n_grams = word_tokenize(a)
    for i in bigrams:
        n_grams.append(' '.join(list(i)))
    for i in trigrams:
        n_grams.append(' '.join(list(i)))    
    return ','.join(n_grams)

In [208]:
df['text_ngrams'] = map(lambda x: return_n_grams(x), df['text_lemmatized'])

In [209]:
df.head(2)

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,...,replies_count,participants_count,likes,comments,shares,type,text_sw,text_char_only,text_lemmatized,text_ngrams
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,print they should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,...,0,1,0,0,0,bias,print pay back money plus interest. entire fam...,print pay back money plus interest entire fami...,print pay back money plus interest entire fami...,"print,pay,back,money,plus,interest,entire,fami..."
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,why did attorney general loretta lynch plead t...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,...,0,1,0,0,0,bias,attorney general loretta lynch plead fifth? ba...,attorney general loretta lynch plead fifth bar...,attorney general loretta lynch plead fifth bar...,"print,pay,back,money,plus,interest,entire,fami..."
